# Demo Analysis

## 1. Setup Workspace

In [1]:
import json
import pathlib
import copy
import operator
import pathlib
import os

import anytree
import networkx as nx
import sympy

import paptree

## 2. Load Data

In [2]:
data_dir = pathlib.Path.cwd().parent / "experiments/demo/data"
raw_data_dir = data_dir / "raw"
trace_file = raw_data_dir / "paptrace.darwin-arm64-clang.json"
trees = paptree.utils.from_file(trace_file)
print(f"Loaded {len(trees)} traces.")

Unique iter blocks: [[264376, 264369, 264367]]

LoopNode(desc='for (int i = 2; i <= std::sqrt(n); ++i)', iter_block=[CallNode(name=264314, params=[{'name': 'n', 'value': '4'}], sig='typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int)', type='CallerExpr'), StmtNode(desc='LoopIter', name=264376, type='LoopIter')], iter_count=1, name=264379, trailing_iter_block=None, type='ForStmt')
├── CallNode(name=264314, params=[{'name': 'n', 'value': '4'}], sig='typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int)', type='CallerExpr')
└── StmtNode(desc='LoopIter', name=264376, type='LoopIter')
    └── StmtNode(desc='n % i == 0', name=264369, type='IfThenStmt')
        └── StmtNode(desc='return false', name=264367, type='ReturnStmt')

Unique iter blocks: [[264376, 264369, 264367]]

LoopNode(desc='for (int i = 2; i <= std::sqrt(n); ++i)', iter_block=[CallNode(name=264314, params=[{'name': 'n', 'value': '6'}], sig='typename std::enable_if<std::is_integr

In [3]:
known_file = raw_data_dir / "known_exprs.json"
with open(known_file, "r") as f_in:
    known_exprs = json.loads(f_in.read())
print(f"Loaded {len(known_exprs)} known expressions.")
print(json.dumps(known_exprs, indent=2))

Loaded 1 known expressions.
{
  "int operator*=(int, int)": "C_OP_MUL_ASSIGN_INT_INT"
}


## 3. Node Summary

### 3.1. Unfiltered Nodes

In [4]:
nodes = []
for tree in trees:
    nodes.extend(anytree.PreOrderIter(tree.root))
print(f"Node count: {len(nodes)}")

Node count: 232


In [5]:
node_types = {}
for node in nodes:
    node_types.setdefault(node.type, []).append(node)
print(f"Node type count: {len(node_types)}")

Node type count: 7


In [6]:
print("Nodes per type:")
for k, v in node_types.items():
    print(f"- {k}: {len(v)}")

Nodes per type:
- CalleeExpr: 23
- ReturnStmt: 23
- IfThenStmt: 9
- ForStmt: 12
- LoopIter: 57
- OpExpr: 92
- CallerExpr: 16


### 3.2. Unique Nodes

In [7]:
unique_nodes = []
for node in nodes:
    if node not in unique_nodes:
        unique_nodes.append(node)
print(f"Unique node count: {len(unique_nodes)}")

Unique node count: 54


In [8]:
# We expect this count to match the unfiltered node type count.
unique_node_types = {}
for node in unique_nodes:
    unique_node_types.setdefault(node.type, []).append(node)
print(f"Node type count: {len(unique_node_types)}")

Node type count: 7


In [9]:
print("Nodes per type:")
for k, v in unique_node_types.items():
    print(f"- {k}: {len(v)}")

Nodes per type:
- CalleeExpr: 23
- ReturnStmt: 5
- IfThenStmt: 3
- ForStmt: 12
- LoopIter: 2
- OpExpr: 3
- CallerExpr: 6


## 4. Trace Summary

In [10]:
binned_trees = {}
for tree in trees:
    binned_trees.setdefault(tree.root.sig, []).append(tree)
print(f"Trace entry point count: {len(binned_trees)}")

Trace entry point count: 3


In [11]:
print("Traces per entry point:")
for k, v in binned_trees.items():
    print(f"- {k}: {len(v)}")

Traces per entry point:
- _Bool demo::IsEven(int): 8
- int demo::Factorial(int): 8
- _Bool demo::IsPrime(int): 7


## 5. Trace Analysis

### 5.1. IsEven

In [12]:
trees_subset = binned_trees["_Bool demo::IsEven(int)"]

def to_params_str(params):
    #return ", ".join([f"{param['name']}={param['value']}" for param in params])
    return ", ".join([f"{param['value']}" for param in params])

def to_call_str(node):
    return f"{node.sig}: ({to_params_str(node.params)})"

def to_simple_node_view(node):
    sym = "sym @ " if hasattr(node, "target") else ""
    if node.is_call_node():
        node_type = type(node).__name__
        desc = f"{node.type}: {to_call_str(node)}"
    else:
        node_type = type(node).__name__
        desc = f"{node.type}: {node.desc}"
    return f"({sym}{node.name}) [{node_type}] {desc}"

for tree in operator.itemgetter(0, 1, 2)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(263487) [CallNode] CalleeExpr: _Bool demo::IsEven(int): (0)
└── (263485) [StmtNode] ReturnStmt: return value % 2 == 0

(263487) [CallNode] CalleeExpr: _Bool demo::IsEven(int): (2)
└── (263485) [StmtNode] ReturnStmt: return value % 2 == 0

(263487) [CallNode] CalleeExpr: _Bool demo::IsEven(int): (4)
└── (263485) [StmtNode] ReturnStmt: return value % 2 == 0



In [13]:
results = paptree.analyze.analyze(known_exprs, trees_subset)
print("\nResults:")
print(json.dumps(results, indent=2))

Path summary:
- _Bool demo::IsEven(int): 1 paths
  - [path_0] (263485,): 8 traces

Finding general expr. for: _Bool demo::IsEven(int): (0)
  Path is constant.
  Found expr.: C_263487

Results:
{
  "sigs": {
    "_Bool demo::IsEven(int)": "sig_0"
  },
  "ctxs": {
    "sig_0": {
      "0": "path_0",
      "2": "path_0",
      "4": "path_0",
      "1": "path_0",
      "3": "path_0",
      "5": "path_0",
      "-2147483648": "path_0",
      "2147483647": "path_0"
    }
  },
  "exprs": {
    "sig_0": {
      "path_0": "Symbol('C_263487')"
    }
  }
}


### 5.2. Factorial

In [14]:
trees_subset = binned_trees["int demo::Factorial(int)"]
for tree in operator.itemgetter(0, 2, 3, 4)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(263691) [CallNode] CalleeExpr: int demo::Factorial(int): (-1)
└── (263577) [StmtNode] IfThenStmt: value < 0 || value > 31
    └── (263575) [StmtNode] ReturnStmt: return -1

(263691) [CallNode] CalleeExpr: int demo::Factorial(int): (0)
├── (263675) [LoopNode] ForStmt: for (int i = 1; i <= value; ++i)
└── (263689) [StmtNode] ReturnStmt: return result

(263691) [CallNode] CalleeExpr: int demo::Factorial(int): (1)
├── (263675) [LoopNode] ForStmt: for (int i = 1; i <= value; ++i)
│   ├── (263672) [StmtNode] LoopIter: LoopIter
│   │   └── (263666) [StmtNode] OpExpr: int operator*=(int, int)
│   └── (263652) [StmtNode] OpExpr: int operator++
└── (263689) [StmtNode] ReturnStmt: return result

(263691) [CallNode] CalleeExpr: int demo::Factorial(int): (2)
├── (263675) [LoopNode] ForStmt: for (int i = 1; i <= value; ++i)
│   ├── (263672) [StmtNode] LoopIter: LoopIter
│   │   └── (263666) [StmtNode] OpExpr: int operator*=(int, int)
│   ├── (263652) [StmtNode] OpExpr: int operator++
│   ├── (26367

In [15]:
results = paptree.analyze.analyze(known_exprs, trees_subset)
print("\nResults:")
print(json.dumps(results, indent=2))

Path summary:
- int demo::Factorial(int): 3 paths
  - [path_0] (263577, 263575): 2 traces
  - [path_1] (263675, 263689): 1 traces
  - [path_2] (263675, 263672, 263689): 5 traces

Finding general expr. for: int demo::Factorial(int): (0)
  Path is constant.
  Found expr.: C_263691

Finding general expr. for: int demo::Factorial(int): (1)
  Path is constant.
  Found expr.: C_263691

Finding general expr. for: int demo::Factorial(int): (2)
  Solving for loop expr for node: 263675
    Setting loop expr for 5358415184 to sqrt(X0**2)
  Found expr.: C_263691 + (C_OP_MUL_ASSIGN_INT_INT + T_263652)*sqrt(X0**2)

Results:
{
  "sigs": {
    "int demo::Factorial(int)": "sig_0"
  },
  "ctxs": {
    "sig_0": {
      "-1": "path_0",
      "32": "path_0",
      "0": "path_1",
      "1": "path_2",
      "2": "path_2",
      "3": "path_2",
      "4": "path_2",
      "31": "path_2"
    }
  },
  "exprs": {
    "sig_0": {
      "path_0": "Symbol('C_263691')",
      "path_1": "Symbol('C_263691')",
      "path

### 5.3. IsPrime

In [16]:
trees_subset = binned_trees["_Bool demo::IsPrime(int)"]
results = paptree.analyze.analyze(known_exprs, trees_subset)
print("\nResults:")
print(json.dumps(results, indent=2))

Getting cf nodes for CallNode(name=264314, params=[{'name': 'n', 'value': '4'}], sig='typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int)', type='CallerExpr')
Getting cf nodes for StmtNode(desc='LoopIter', name=264376, type='LoopIter')

Getting cf nodes for CallNode(name=264314, params=[{'name': 'n', 'value': '6'}], sig='typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int)', type='CallerExpr')
Getting cf nodes for StmtNode(desc='LoopIter', name=264376, type='LoopIter')

Getting cf nodes for CallNode(name=264314, params=[{'name': 'n', 'value': '9'}], sig='typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int)', type='CallerExpr')
Getting cf nodes for StmtNode(desc='LoopIter', name=264376, type='LoopIter')
Getting cf nodes for StmtNode(desc='int operator++', name=264336, type='OpExpr')

Getting cf nodes for CallNode(name=264314, params=[{'name': 'n', 'value': '25'}], sig='typename std::enable_if<std::is_integral<in

In [17]:
# Path 0
for tree in operator.itemgetter(0, 1)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(264390) [CallNode] CalleeExpr: _Bool demo::IsPrime(int): (1)
└── (263759) [StmtNode] IfThenStmt: n <= 1
    └── (263757) [StmtNode] ReturnStmt: return false

(264390) [CallNode] CalleeExpr: _Bool demo::IsPrime(int): (4)
└── (264379) [LoopNode] ForStmt: for (int i = 2; i <= std::sqrt(n); ++i)
    ├── (264314) [CallNode] CallerExpr: typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int): (4)
    └── (264376) [StmtNode] LoopIter: LoopIter
        └── (264369) [StmtNode] IfThenStmt: n % i == 0
            └── (264367) [StmtNode] ReturnStmt: return false



In [18]:
# Path 1
for tree in operator.itemgetter(2, 3, 4)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(264390) [CallNode] CalleeExpr: _Bool demo::IsPrime(int): (6)
└── (264379) [LoopNode] ForStmt: for (int i = 2; i <= std::sqrt(n); ++i)
    ├── (264314) [CallNode] CallerExpr: typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int): (6)
    └── (264376) [StmtNode] LoopIter: LoopIter
        └── (264369) [StmtNode] IfThenStmt: n % i == 0
            └── (264367) [StmtNode] ReturnStmt: return false

(264390) [CallNode] CalleeExpr: _Bool demo::IsPrime(int): (9)
└── (264379) [LoopNode] ForStmt: for (int i = 2; i <= std::sqrt(n); ++i)
    ├── (264314) [CallNode] CallerExpr: typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int): (9)
    ├── (264376) [StmtNode] LoopIter: LoopIter
    ├── (264336) [StmtNode] OpExpr: int operator++
    ├── (264314) [CallNode] CallerExpr: typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int): (9)
    └── (264376) [StmtNode] LoopIter: LoopIter
        └── (264369) [StmtNode] IfThenStmt: n % i ==

In [19]:
# Path 2
for tree in operator.itemgetter(5, 6)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(264390) [CallNode] CalleeExpr: _Bool demo::IsPrime(int): (57)
└── (264379) [LoopNode] ForStmt: for (int i = 2; i <= std::sqrt(n); ++i)
    ├── (264314) [CallNode] CallerExpr: typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int): (57)
    ├── (264376) [StmtNode] LoopIter: LoopIter
    ├── (264336) [StmtNode] OpExpr: int operator++
    ├── (264314) [CallNode] CallerExpr: typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int): (57)
    └── (264376) [StmtNode] LoopIter: LoopIter
        └── (264369) [StmtNode] IfThenStmt: n % i == 0
            └── (264367) [StmtNode] ReturnStmt: return false

(264390) [CallNode] CalleeExpr: _Bool demo::IsPrime(int): (91)
└── (264379) [LoopNode] ForStmt: for (int i = 2; i <= std::sqrt(n); ++i)
    ├── (264314) [CallNode] CallerExpr: typename std::enable_if<std::is_integral<int>::value, double>::type sqrt(int): (91)
    ├── (264376) [StmtNode] LoopIter: LoopIter
    ├── (264336) [StmtNode] OpExpr: int operato

In [20]:
# Path 3
for tree in operator.itemgetter(10, 11)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

IndexError: list index out of range

In [ ]:
# Path 4?
for tree in operator.itemgetter(12, 13, 14)(trees_subset):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()